In [ ]:
#!pip install raiwidgets
#!pip install --upgrade raiwidgets

In [12]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai import RAIInsights 

import shap 
import sklearn 
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import load_diabetes

In [13]:
data = load_diabetes(as_frame=True)

In [14]:
target_features = "y"
continuous_features = data.feature_names
data_df = pd.DataFrame(data.data, columns=data.feature_names)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(data_df, data.target, test_size=0.2, random_state=0)

train_data = X_train.copy()
test_data = X_test.copy()
train_data[target_features] = y_train
test_data[target_features] = y_test
data.feature_names

['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

In [16]:
model = RandomForestRegressor().fit(X_train, y_train)

In [17]:
rai_insights = RAIInsights(model, train_data, test_data, target_features, "regression", categorical_features=[])

In [18]:
# Interpretability 
rai_insights.explainer.add()

# Error analysis
rai_insights.error_analysis.add()

# Counrterfactuals: accepts total number of counterfactuals to generate
# and a list of categorical feature names 
rai_insights.counterfactual.add(total_CFs=20, desired_range=[50, 120])

In [ ]:
rai_insights.compute()

In [ ]:
ResponsibleAIDashboard(rai_insights)

Fairlearn dashboard

In [29]:
from raiwidgets import FairnessDashboard

from fairlearn.reductions import GridSearch, DemographicParity
from fairlearn.metrics import MetricFrame, selection_rate 

from sklearn import svm, neighbors, tree 
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import fetch_openml 

In [30]:
data = fetch_openml(data_id=1590, as_frame=True)

In [33]:
X_raw = data.data
y_true = (data.target == ">50K") * 1

X_raw["race"].value_counts().to_dict()

{'White': 41762,
 'Black': 4685,
 'Asian-Pac-Islander': 1519,
 'Amer-Indian-Eskimo': 470,
 'Other': 406}

In [32]:
sensitive_features = X_raw[["sex", "race"]]
X = X_raw.drop(labels=["sex", "race"], axis=1)
X = pd.get_dummies(X)

sc = StandardScaler()
X_scaled = sc.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

le = LabelEncoder()
y = le.fit_transform(y_true)

In [34]:
X_train, X_test, y_train, y_test, sensitive_features_train, sensitive_features_test = train_test_split(X_scaled, y_true, sensitive_features, test_size=0.2, random_state=0, stratify=y_true)

In [35]:
# Work around indexing bug
X_train = X_train.reset_index(drop=True)
sensitive_features_train = sensitive_features_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
sensitive_features_test = sensitive_features_test.reset_index(drop=True)

In [36]:
unmitigated_predictor = LogisticRegression(solver='liblinear', fit_intercept=True).fit(X_train, y_train)

In [37]:
y_pred = unmitigated_predictor.predict(X_test)

In [ ]:
FairnessDashboard(sensitive_features=sensitive_features_test, y_true=y_test, y_pred=y_pred)